In [8]:
# Visão Computacional
# Trabalho 1 - Capa invisível

# Equipe: 
#   - Geovani Lopes
#   - Hisrael Braga
#   - Ronald Boadana


import cv2
import time
import numpy as np

cap = cv2.VideoCapture(1)

# captura o primeiro frame para ser utilizado como background
_, background = cap.read()
time.sleep(2)
_, background = cap.read()

cv2.namedWindow("janela")

def mause(pos):
    pass

# Trackbar utilizada para escolher a cor da máscara
cv2.createTrackbar("Color", "janela" , 0, 2,mause)

open_kernel = np.ones((5,5),np.uint8)
close_kernel = np.ones((7,7),np.uint8)
dilation_kernel = np.ones((10, 10), np.uint8)

# Função pra remover o ruído da máscara
def filter_mask(mask):

    close_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, close_kernel)
    open_mask = cv2.morphologyEx(close_mask, cv2.MORPH_OPEN, open_kernel)
    dilation = cv2.dilate(open_mask, dilation_kernel, iterations= 1)

    return dilation


# Loop da câmera
while cap.isOpened():
    ret, frame = cap.read()  # Leitura dos frames da imagem

    color = cv2.getTrackbarPos("Color", 'janela') # Obtenção do valor de cor a ser utilizado na máscara, 0 = verde, 1 = azul, 2 = vermelho

    # Converter a imagem para HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Limites inferiores e superiores pra cor verde 
    lower_bound = np.array([50, 80, 50])     
    upper_bound = np.array([90, 255, 255])
    
    # Limites inferiores e superiores pra cor azul
    lower_bound_blue = np.array([89, 110, 110])     
    upper_bound_blue = np.array([255, 255, 255])

    # Limites inferiores e superiores pra cor vermelha
    lower_bound_red = np.array([50, 50, 80])     
    upper_bound_red = np.array([255, 255, 255])


    # Aplicar a máscara de acordo com a cor escolhida
    if(color == 0):  # Cor verde
        mask = cv2.inRange(hsv, lower_bound, upper_bound)
    elif(color == 1): # Cor azul
        mask = cv2.inRange(hsv, lower_bound_blue, upper_bound_blue)
    else:  # Cor vermelha
        mask = cv2.inRange(hsv, lower_bound_red, upper_bound_red)
            
    # Filtrando a máscara
    mask = filter_mask(mask)

    # Aplicação da máscara utilizando bitwise
    cloak = cv2.bitwise_and(background, background, mask=mask)

    # Criação de uma máscara inversa
    inverse_mask = cv2.bitwise_not(mask)  

    # Aplicação da máscara inversa na imagem atual
    current_background = cv2.bitwise_and(frame, frame, mask=inverse_mask)

    # Combinando as imagens
    combined = cv2.add(cloak, current_background)

    # Exibindo a imagem gerada com a capa invisível
    cv2.imshow("Capa invisível", combined)

    # Encerrar
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()